In [1]:
!pip install selenium
!pip install undetected-chromedriver


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
import pandas as pd
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [28]:
# парсинг по листам объявлений
start_time = time.time()
url = "https://krym.cian.ru/kupit-kvartiru/"

driver = uc.Chrome()
driver.get(url)

cookie_notification = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, ".//div[@data-name='CookiesNotification']"))
)

# Закрыть уведомление о файлах cookie, если оно видимо
if cookie_notification.is_displayed():
    cookie_button = cookie_notification.find_element(By.CLASS_NAME, "_25d45facb5--button--KVooB._25d45facb5--button--gs5R_._25d45facb5--M--I5Xj6._25d45facb5--button--DsA7r")
    cookie_button.click()

# Переключиться на основное окно
driver.switch_to.default_content()

In [30]:
from selenium.webdriver import Keys

squares = []
layouts = []
floors = []
links=[]
addresses = []
main_prices = []
sq_prices = []
descriptions = []
lost_descriptions = []
broken_title = []
while True:
    time.sleep(2)
    # Извлечение данных объявлений
    ads = driver.find_elements(By.XPATH, ".//article[@data-name='CardComponent']")

    for ad in ads:
        title = ad.find_element(By.XPATH, ".//span[@data-mark='OfferTitle']").text
        try:

            layout = title[:title.index(',')].strip()
            square = title[title.index(',')+1:title.index('м²,')].strip()
            floor = title[title.index('м²,')+3:].strip()
        except:
            link = ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href')
            print(title)
            broken_title.append(link)
            continue
        layouts.append(layout)
        squares.append(square)
        floors.append(floor)
        links.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--link--eoxce").get_attribute('href'))

        addresses.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--labels--L8WyJ").text)

        main_prices.append(ad.find_element(By.XPATH, ".//span[@data-mark='MainPrice']").text)

        sq_prices.append(ad.find_element(By.XPATH, ".//p[@data-mark='PriceInfo']").text)

        try:
            descriptions.append(ad.find_element(By.CLASS_NAME, "_93444fe79c--description--SqTNp").text)
        except NoSuchElementException:
            descriptions.append('')
            lost_descriptions.append(len(addresses))
    # Проверка наличия кнопки "Дальше"
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "_93444fe79c--button--Cp1dl") and contains(@class, "_93444fe79c--link-button--Pewgf") and contains(@class, "_93444fe79c--M--T3GjF") and contains(@class, "_93444fe79c--button--dh5GL") and span[@class="_93444fe79c--text--rH6sj" and text()="Дальше"]]')
    except Exception as e:
        print(e)
        break

    body = driver.find_element(By.TAG_NAME, "body")
    # Кликнуть на кнопку "Дальше" для перехода на следующую страницу
    next_button.click()
len(broken_title)

MaxRetryError: HTTPConnectionPool(host='localhost', port=55441): Max retries exceeded with url: /session/a10a385e9f87c5c41f11b60329945155/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4abe088b20>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [29]:
print(len(squares),len(layouts),len(floors),len(addresses),len(main_prices),len(sq_prices),len(descriptions))
data = {
    'squares' : squares,
    'layouts' : layouts,
    'floors' : floors,
    'address': addresses,
    'main_price': main_prices,
    'sq_price': sq_prices,
    'links': links,
    'description': descriptions,
}
driver.quit()
end_time = time.time()
execution_time = end_time - start_time
print(f"{len(squares)} объявлений обработаны за {execution_time} секунд.")
df = pd.DataFrame(data)
df

605 605 605 605 605 605 605
605 объявлений обработаны за 114.20099186897278 секунд.


,squares,layouts,floors,address,main_price,sq_price,links,description
0,"33,7",1-комн. кв.,3/8 этаж,"Республика Крым, Феодосия, Утесов ЖК",4 381 000 ₽,130 000 ₽/м²,https://krym.cian.ru/sale/flat/274690246/,
1,"37,7",1-комн. кв.,3/9 этаж,"Республика Крым, Евпатория, проспект Победы, 8...",4 350 000 ₽,115 385 ₽/м²,https://krym.cian.ru/sale/flat/287297423/,СРОЧНАЯ ПРОДАЖА Продается шикарная - просторна...
2,"29,3",1-комн. кв.,4/5 этаж,"Республика Крым, Феодосия городской округ, Орд...",4 400 000 ₽,150 171 ₽/м²,https://krym.cian.ru/sale/flat/286560661/,В продаже уютная однокомнатная квартира в куро...
3,"44,2",2-комн. кв.,3/5 этаж,"Республика Крым, Джанкой, улица Толстого",4 500 000 ₽,101 810 ₽/м²,https://krym.cian.ru/sale/flat/286554975/,Двухкомнатная квартира для Вашей семьи в Джанк...
4,"31,5",2-комн. кв.,1/1 этаж,"Республика Крым, Симферополь, р-н Железнодорож...",4 500 000 ₽,142 857 ₽/м²,https://krym.cian.ru/sale/flat/287178223/,Отличный вариант для жизни и сдачи в аренду! Д...
...,...,...,...,...,...,...,...,...
600,24,1-комн. кв.,1/5 этаж,"Республика Крым, Саки, улица Строительная, 16",3 600 000 ₽,150 000 ₽/м²,https://krym.cian.ru/sale/flat/284926866/,Предлагается к продаже однокомнатная квартира ...
601,34,1-комн. кв.,4/6 этаж,"Республика Крым, Алушта, улица Октябрьская, 43",4 500 000 ₽,132 353 ₽/м²,https://krym.cian.ru/sale/flat/281601789/,Продается светлая и просторная 1-но комнатная ...
602,"17,6",1-комн. кв.,1/4 этаж,"Республика Крым, Алушта, улица Туристов, 1А",1 700 000 ₽,96 591 ₽/м²,https://krym.cian.ru/sale/flat/288084669/,Объект 58492 В продаже 1-комнатная квартира в ...
603,"30,4",1-комн. кв.,1/2 этаж,"Республика Крым, Саки, улица Гайнутдинова, 3",3 900 000 ₽,128 289 ₽/м²,https://krym.cian.ru/sale/flat/284603093/,Продам 1к квартиру в г. Саки! Продается отличн...


In [16]:
# парсинг по каждому объявлению
latitude = []
longitude = []
dates = []
total_area = []
addresses = []
main_prices = []
sq_prices = []
descriptions = []


start_time = time.time()
driver = uc.Chrome()
i = 200
links1 = links + broken_title
print(len(links1))
for link in links+broken_title:
    try:
        driver.get(link)
        # latitude =
        # longitude =

        dates.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--color_gray40_100--ppbi0.a10a3f92e9--lineHeight_5u--cJ35s.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__0--mdnqq").text)


        data1 = driver.find_elements(By.XPATH, ".//div[@data-name='ObjectFactoidsItem']")

        total_area.append(data1[0].text.split('\n')[1])


        addresses.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--address-line--GRDTb").text[:-8])
        main_prices.append(driver.find_element(By.XPATH, ".//div[@data-name='PriceInfo']").text)
        data2 = driver.find_elements(By.CLASS_NAME, "a10a3f92e9--color_black_100--kPHhJ.a10a3f92e9--lineHeight_20px--tUURJ.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__normal--xbqP6")
        sq_prices.append(data2[1].text)
        descriptions.append(driver.find_element(By.CLASS_NAME, "a10a3f92e9--color_black_100--kPHhJ.a10a3f92e9--lineHeight_6u--A1GMI.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_16px--RB9YW.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9--text_letterSpacing__0--mdnqq.a10a3f92e9--text_whiteSpace__pre-wrap--scZwb").text)
    except Exception as e:
        print()
        print(e)
        print(link)
    i-=1
    if i ==0:
        break
    if i %10 == 0:
        print(i)
driver.quit()


1537
190
180
170
160
150
140
130
120
110
100
90
80
70
60
50
40
30
20
10


In [17]:
print(len(dates))
data = {
    # 'latitude': latitude,
    # 'longitude': longitude,
    'dates' : dates,
    'total_area' : total_area,
    'addresses' : addresses,
    'main_prices' : main_prices,
    'sq_prices' : sq_prices,
    'descriptions' : descriptions,
}
driver.quit()
end_time = time.time()
execution_time = end_time - start_time
print(f"{len(dates)} объявлений обработаны за {execution_time} секунд.")
df = pd.DataFrame(data)
df

200
200 объявлений обработаны за 379.5182251930237 секунд.


,dates,total_area,addresses,main_prices,sq_prices,descriptions
0,"Обновлено: 5 июн, 18:16","197,09 м²","Крым респ., Алушта, Профессорский Уголок микро...",40 032 527 ₽,203 118 ₽/м²,Продаётся 3-комнатная квартира в сданном доме ...
1,"Обновлено: 5 июн, 18:46","48,39 м²","Крым респ., Алушта, Профессорский Уголок микро...",9 529 946 ₽,196 940 ₽/м²,Продаётся 2-комнатная квартира в строящемся до...
2,"Обновлено: 23 май, 11:06",25 м²,"Крым респ., Бахчисарайский район, Песчаное сел...",1 800 000 ₽,72 000 ₽/м²,"Апартаменты в цоколе 25 м2, трехэтажного дома...."
3,"Обновлено: 5 июн, 18:46","60,71 м²","Крым респ., Алушта, ул. Западная, 4к3",8 900 000 ₽,146 599 ₽/м²,Продаются 1-комнатные апартаменты в сданном до...
4,"Обновлено: 23 май, 11:07","65,1 м²","Крым респ., Бахчисарайский район, Песчаное сел...",5 300 000 ₽,81 413 ₽/м²,"Трехкомнатная квартира 65,1 м2 на этаже 2/2 до..."
...,...,...,...,...,...,...
195,"Обновлено: 3 июн, 04:54",32 м²,"Крым респ., Симферополь, р-н Железнодорожный, ...",4 200 000 ₽,131 250 ₽/м²,Лучшее решение для вас в одном из самых привле...
196,"Обновлено: сегодня, 05:19",130 м²,"Крым респ., Ялта городской округ, Гурзуф пгт, ...",31 000 000 ₽,238 462 ₽/м²,Квартира 801 на 9 этаже комплекса с собственно...
197,"Обновлено: 30 май, 14:59",20 м²,"Крым респ., Алушта, пер. Базарный, 1",3 700 000 ₽,185 000 ₽/м²,МЫ НЕ БЕРЕМ КОМИССИЮ С ПОКУПАТЕЛЯ!!!\nПредлага...
198,"Обновлено: сегодня, 05:05","87,17 м²","Крым респ., Симферопольский район, Мирное село...",8 542 660 ₽,98 000 ₽/м²,Продаётся 3-комнатная квартира в строящемся до...


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

dates = []
total_area = []
addresses = []
main_prices = []
sq_prices = []
descriptions = []

start_time = time.time()

for link in links:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        dates.append(soup.find('div', class_='a10a3f92e9--color_gray40_100--ppbi0.a10a3f92e9--lineHeight_5u--cJ35s'
                                             '.a10a3f92e9--fontWeight_normal--P9Ylg.a10a3f92e9--fontSize_14px--TCfeJ'
                                             '.a10a3f92e9--display_block--pDAEx.a10a3f92e9--text--g9xAG.a10a3f92e9'
                                             '--text_letterSpacing__0--mdnqq').text)

        data1 = soup.find_all('div', {'data-name': 'ObjectFactoidsItem'})
        total_area.append(data1[0].find_all('span')[1].text)

        addresses.append(soup.find('div', class_='a10a3f92e9--address-line--GRDTb').text[:-8])
        main_prices.append(soup.find('div', {'data-name': 'PriceInfo'}).text)

        data2 = soup.find_all('span', class_='a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_20px--tUURJ '
                                             'a10a3f92e9--fontWeight_normal--P9Ylg a10a3f92e9--fontSize_14px--TCfeJ '
                                             'a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG '
                                             'a10a3f92e9--text_letterSpacing__normal--xbqP6')
        sq_prices.append(data2[1].text)

        descriptions.append(soup.find('span', class_='a10a3f92e9--color_black_100--kPHhJ '
                                                     'a10a3f92e9--lineHeight_6u--A1GMI '
                                                     'a10a3f92e9--fontWeight_normal--P9Ylg '
                                                     'a10a3f92e9--fontSize_16px--RB9YW '
                                                     'a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG '
                                                     'a10a3f92e9--text_letterSpacing__0--mdnqq '
                                                     'a10a3f92e9--text_whiteSpace__pre-wrap--scZwb').text)
    except Exception as e:
        print()
        print(e)
        print(link)

In [ ]:
data = {
    'dates': dates,
    'total_area': total_area,
    'addresses': addresses,
    'main_prices': main_prices,
    'sq_prices': sq_prices,
    'descriptions': descriptions,
}

end_time = time.time()
execution_time = end_time - start_time
print(f"{len(dates)} объявлений обработаны за {execution_time} секунд.")

df = pd.DataFrame(data)
print(df)